## Introduction:
You can use this ipython notebook as a template for the rest of the homework.

### 0. Basic Useful Setups:

In [1]:
### Basic setups
import sys
sys.path.append('./models/')
# Enable automatic reload of libraries
%load_ext autoreload
# All modules are reloaded before every comment
%autoreload 2
import keras

/opt/conda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### 1. Read MNIST using Keras

In [2]:
from utils import load_mnist
(x_train, y_train), (x_test, y_test) = load_mnist()

('x_train shape:', (60000, 28, 28, 1))
(60000, 'train samples')
(10000, 'test samples')


## 2. Define VGG5 Model

In [3]:
from keras.layers import (
    Conv2D, BatchNormalization, Activation,
    MaxPooling2D, Dense, Flatten
)

def vgg(input_tensor):
    def two_conv_pool(x, F1, F2, name):
        x = Conv2D(F1, (3, 3), activation=None, padding='same', name='{}_conv1'.format(name))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(F2, (3, 3), activation=None, padding='same', name='{}_conv2'.format(name))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((2, 2), strides=(2, 2), name='{}_pool'.format(name))(x)

        return x

    def three_conv_pool(x, F1, F2, F3, name):
        x = Conv2D(F1, (3, 3), activation=None, padding='same', name='{}_conv1'.format(name))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(F2, (3, 3), activation=None, padding='same', name='{}_conv2'.format(name))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(F3, (3, 3), activation=None, padding='same', name='{}_conv3'.format(name))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((2, 2), strides=(2, 2), name='{}_pool'.format(name))(x)

        return x
    net = input_tensor

    net = two_conv_pool(net, 32, 32, "block1")
    net = two_conv_pool(net, 64, 64, "block2")
    net = three_conv_pool(net, 128, 128, 128, "block3")
    net = three_conv_pool(net, 256, 256, 256, "block4")

    net = Flatten()(net)
    net = Dense(512, activation='relu', name='fc-1')(net)
    net = Dense(512, activation='relu', name='fc-2')(net)
    net = Dense(10, activation='softmax', name='predictions')(net)

    return net

### 2. Load VGG-5 Model

In [4]:
from keras.models import Model
from keras.layers import Input
X = Input(shape=[28, 28, 1])
y = vgg(X)
model = Model(X, y, name="vgg")
model.compile("adam", "categorical_crossentropy", ["accuracy"])


### 2. Train and Evaluate VGG5 Model

In [5]:
# train model
BATCH_SIZE = 128
MAX_EPOCH = 12

hist = model.fit(x_train, y_train, validation_data=(x_test, y_test),batch_size=BATCH_SIZE, nb_epoch=12, verbose=1)

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 60000 samples, validate on 10000 samples
Epoch 1/12


ResourceExhaustedError: OOM when allocating tensor of shape [3,3,128,256] and type float
	 [[Node: training/Adam/zeros_28 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [3,3,128,256] values: [[[0 0 0]]]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op u'training/Adam/zeros_28', defined at:
  File "/opt/conda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/opt/conda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/opt/conda/lib/python2.7/site-packages/tornado/ioloop.py", line 1064, in start
    ha

In [ ]:
score = cnn.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])